# iNaturalist status updates by state - VIC

Using the file produced in the collate-status-taxa.ipynb: `inat-aust-status-taxa.csv`, generate lists to update iNaturalist statuses

## Prep - common to all states
1. Read in the inaturalist statuses & filter out this state
2. Read in the inaturalist taxa list
3. Read in the state sensitive and conservation list
4. Attempt to match the state statuses to an IUCN equivalent
5. Determine the best placeID to use for this state

**Next steps:**
Establish the changes that need to be made. Read in the sensitive and conservation list for each state.
    a. new - any new species that appear in the state lists but do not have a status in inaturalist (new template)
    b. updates - any changes to information which was added by us previously (user_id = 708886) (update template, action='UPDATE')
    c. removals - any statuses which were added by us previously (user_id = 708886) list which are incorrect (update template, action='REMOVE')
    d. flags - are there any statuses by other users that need to be flagged?

### 1. iNaturalist statuses

In [261]:
import pandas as pd

projectdir = "/Users/oco115/PycharmProjects/authoritative-lists/" # basedir for this gh project
# projectdir = "/Users/new330/IdeaProjects/authoritative-lists/" # basedir for this gh project
sourcedir = projectdir + "source-data/inaturalist-statuses/"
listdir = projectdir + "current-lists/"

# read in the statuses
taxastatus = pd.read_csv(sourcedir + "inat-aust-status-taxa.csv", encoding='UTF-8',na_filter=False,dtype=str) ## Read inaturalist conservation statuses file
taxastatus.head(3)


,id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
0,166449,38493,1138587,7830,,Flora and Fauna Guarantee Act 1988,CR,,,obscured,...,Eulamprus,kosciuskoi,,2021-03-01T10:35:01Z,Eulamprus kosciuskoi,species,http://reptile-database.reptarium.cz/search.ph...,,,
1,234788,918383,702203,9994,,Atlas of Living Australia,NT,https://bie.ala.org.au/species/https://id.biod...,,,...,Chiloschista,phyllorhiza,,2022-01-08T03:30:36Z,Chiloschista phyllorhiza,species,http://www.catalogueoflife.org/annual-checklis...,,,
2,234789,918383,702203,7308,,Atlas of Living Australia,LC,https://bie.ala.org.au/species/https://id.biod...,,,...,Chiloschista,phyllorhiza,,2022-01-08T03:30:36Z,Chiloschista phyllorhiza,species,http://www.catalogueoflife.org/annual-checklis...,,,


In [262]:
def filter_state_statuses(stateregex: str, urlregex: str):
    authoritydf = taxastatus['authority'].drop_duplicates().sort_values()
    authoritydf = authoritydf[pd.Series(authoritydf).str.contains(stateregex)]
    urldf = taxastatus['url'].drop_duplicates().sort_values()
    urldf = urldf[pd.Series(urldf).str.contains(urlregex)]
    placedisplaydf = taxastatus['place_display_name'].drop_duplicates().sort_values()
    placedisplaydf = placedisplaydf[pd.Series(placedisplaydf).str.contains(stateregex)]
    placedf = taxastatus['place_name'].drop_duplicates().sort_values()
    placedf = placedf[pd.Series(placedf).str.contains(stateregex)]
    # concat all and remove duplicates
    statedf = pd.concat([taxastatus.apply(lambda row: row[taxastatus['place_display_name'].isin(placedisplaydf)]),
                         taxastatus.apply(lambda row: row[taxastatus['place_name'].isin(placedf)]),
                         taxastatus.apply(lambda row: row[taxastatus['url'].isin(urldf)]),
                         taxastatus.apply(
                             lambda row: row[taxastatus['authority'].isin(authoritydf)])]).drop_duplicates()
    return statedf.sort_values(['taxon_id', 'user_id'])

inatstatuses = filter_state_statuses(" SA |South Australia|SOUTH AUSTRALIA","sa.gov.au")
inatstatuses.rename(columns={'id':'status_id','id_y':'taxon_id_y'},inplace=True)
inatstatuses

,status_id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
147,152333,100616,708886,6899,16651,SA DEWNR,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Euastacus,bispinosus,,2022-06-06T16:26:39Z,Euastacus bispinosus,species,http://www.iucnredlist.org/apps/redlist/details,,,
89,180750,1038978,624851,6899,,"Department of Environment and Water, Governmen...",Endangered,,Known from just a few areas and near human dis...,obscured,...,Hibbertia,hirsuta,,2021-08-31T12:37:27Z,Hibbertia hirsuta,species,,,,
2631,163879,1094266,708886,6899,16651,SA DEWNR,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Lagenophora,sublyrata,,2020-11-16T20:50:54Z,Lagenophora sublyrata,species,,,,
389,223419,1104996,624851,6899,,"Department of Environment and Water, Governmen...",E,,Endangered in South Australia because it known...,obscured,...,Ptilotus,angustifolius,,2021-10-17T07:32:41Z,Ptilotus angustifolius,species,https://eol.org/pages/55577306,,,
922,152427,116999,,6899,16651,SA DEWNR,endangered,https://lists.ala.org.au/speciesListItem/list/...,,,...,Pandion,haliaetus,,2022-06-27T03:09:49Z,Pandion haliaetus,species,http://www.birdlife.org/datazone/speciesfactsh...,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,170156,892980,702203,6899,,"Department of Environment and Heritage, South ...",Threatened,https://www.landscape.sa.gov.au › threatened_p...,,,...,Daviesia,pectinata,,2021-07-27T12:40:15Z,Daviesia pectinata,species,http://www.catalogueoflife.org/annual-checklis...,,,
1257,152426,892983,708886,6899,16651,SA DEWNR,Endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Caladenia,lowanensis,,2022-10-22T05:13:46Z,Caladenia lowanensis,species,http://www.catalogueoflife.org/annual-checklis...,,,
468,152359,894302,708886,6899,16651,SA DEWNR,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Caladenia,fragrantissima,,2022-06-07T15:06:33Z,Caladenia fragrantissima,species,http://www.catalogueoflife.org/annual-checklis...,,,
245,170258,950220,702203,6899,,Atlas of Living Australia,EN,https://bie.ala.org.au/species/https://id.biod...,,obscured,...,Caladenia,colorata,,2022-06-09T14:15:02Z,Caladenia colorata,species,http://www.catalogueoflife.org/annual-checklis...,,,


### 2. iNaturalist taxonomy

In [263]:
# Output files contain these fields
# Taxon_Name,Status,Authority,IUCN_equivalent,Description,iNaturalist_Place_ID,url,Taxon_Geoprivacy,Username,taxon_id
# so we need to match species from the state lists to the inat taxa to get the taxon_id

import zipfile
url = "https://www.inaturalist.org/taxa/inaturalist-taxonomy.dwca.zip"
filename = url.split("/")[-1]

z=zipfile.ZipFile(sourcedir + filename)

with z.open('taxa.csv') as from_archive:
    inattaxa = pd.read_csv(from_archive,dtype=str)
z.close()
inattaxa.head(3)


,id,taxonID,identifier,parentNameUsageID,kingdom,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references
0,1,https://www.inaturalist.org/taxa/1,https://www.inaturalist.org/taxa/1,https://www.inaturalist.org/taxa/48460,Animalia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-02T06:05:44Z,Animalia,kingdom,http://www.catalogueoflife.org/annual-checklis...
1,2,https://www.inaturalist.org/taxa/2,https://www.inaturalist.org/taxa/2,https://www.inaturalist.org/taxa/1,Animalia,Chordata,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-23T00:40:18Z,Chordata,phylum,http://www.catalogueoflife.org/annual-checklis...
2,3,https://www.inaturalist.org/taxa/3,https://www.inaturalist.org/taxa/3,https://www.inaturalist.org/taxa/355675,Animalia,Chordata,Aves,NaN,NaN,NaN,NaN,NaN,2022-12-27T07:33:16Z,Aves,class,http://www.catalogueoflife.org/annual-checklis...


### 3. State lists

Get the ALA Conservation and Sensitive lists


In [264]:
# %%script echo skipping # comment this line to download dataset from lists.ala.org.au the web and save locally

import sys
import os
sys.path.append(os.path.abspath(projectdir + "source-code/includes"))
import list_functions as lf
# no sensitive list in lists-test
sensitivelist = lf.download_ala_list("https://lists.ala.org.au/ws/speciesListItems/dr884?max=10000&includeKVP=true")
sensitivelist = lf.kvp_to_columns(sensitivelist)
sensitivelist.to_csv(sourcedir + "sa-ala-sensitive.csv", index=False)

conservationlist = lf.download_ala_list("https://lists.ala.org.au/ws/speciesListItems/dr653?max=10000&includeKVP=true")
conservationlist = lf.kvp_to_columns(conservationlist)
conservationlist.to_csv(sourcedir + "sa-ala-conservation.csv", index=False)


In [338]:
# Read sensitive list data
sensitivelist = pd.read_csv(sourcedir + "sa-ala-sensitive.csv", dtype=str)
sensitivelist['sa_geoprivacy'] = 'obscured'
sensitivelist['status']= 'sensitive'
sensitivelist = sensitivelist.rename(columns={'NSXCODE':'taxonID'})
sensitivelist

,id,name,commonName,scientificName,lsid,dataResourceUid,kvpValues,Grouping,SPECIESTYPE,taxonID,vernacular name,Clarifier,Threaten a species/environment;,Threaten someones privacy;,Reason,sa_geoprivacy,status
0,24233,Tiliqua adelaidensis,Pygmy Bluetongue,Tiliqua adelaidensis,https://biodiversity.org.au/afd/taxa/a70c19ab-...,dr884,"[{'key': 'Grouping', 'value': 'Reptiles'}, {'k...",Reptiles,R,Y02572,Pygmy Blue-tongue Lizard (listed as Pygmy Blue...,All known location and burrow records,y,y,The Pygmy Blue-tongue Lizard is endemic to SA ...,obscured,sensitive
1,24377,Calyptorhynchus banksii graptogyne,Red-tailed Black Cockatoo (south-east),Calyptorhynchus (Calyptorhynchus) banksii grap...,https://biodiversity.org.au/afd/taxa/f8ecc7e3-...,dr884,"[{'key': 'Grouping', 'value': 'Cockatoos'}, {'...",Cockatoos,B,Y09312,Red-tailed Black-Cockatoo (south east subspecies),All known nest hollows and artificial nest boxes.,y,NaN,These Cockatoo species are listed under the EP...,obscured,sensitive
2,24336,Calyptorhynchus lathami halmaturinus,Glossy Black-cockatoo (kangaroo Island Subspec...,Calyptorhynchus (Calyptorhynchus) lathami halm...,https://biodiversity.org.au/afd/taxa/d2f5121c-...,dr884,"[{'key': 'Grouping', 'value': 'Cockatoos'}, {'...",Cockatoos,B,C00265,Glossy Black-Cockatoo (Kangaroo Island subspec...,All known nest hollows and artificial nest boxes.,y,NaN,These Cockatoo species are listed under the EP...,obscured,sensitive
3,24331,Lophochroa leadbeateri,Major Mitchell's Cockatoo,Lophochroa leadbeateri,https://biodiversity.org.au/afd/taxa/5815e99d-...,dr884,"[{'key': 'Grouping', 'value': 'Cockatoos'}, {'...",Cockatoos,B,U00270,Major Mitchell's Cockatoo (listed as Cacatua l...,All known nest hollows and artificial nest boxes.,y,NaN,These Cockatoo species are listed under the EP...,obscured,sensitive
4,24267,Calyptorhynchus funereus,Yellow-tailed Black-cockatoo,Zanda funerea,https://biodiversity.org.au/afd/taxa/145b081d-...,dr884,"[{'key': 'Grouping', 'value': 'Cockatoos'}, {'...",Cockatoos,B,G00267,Yellow-tailed Black Cockatoo,All known nest hollows and artificial nest boxes.,y,NaN,some associated subspecies records are sensitive,obscured,sensitive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,4221188,Ixodia achillaeoides subsp. arenicola,NaN,Ixodia achillaeoides subsp. arenicola,https://id.biodiversity.org.au/node/apni/2916083,dr884,"[{'key': 'Grouping', 'value': 'Garden Trade -O...",Garden Trade -Other Ornamental Horticulture pl...,P,ALA concept,NaN,"Grid references, lats and longs, detailed loca...",y,NaN,All species are listed as either Endangered or...,obscured,sensitive
247,4221189,Asplenium bulbiferum subsp. gracillimum,Mother Spleenwort,Asplenium bulbiferum subsp. gracillimum,https://id.biodiversity.org.au/node/apni/2910577,dr884,"[{'key': 'Grouping', 'value': 'Ferns & Allies'...",Ferns & Allies,P,ALA concept,NaN,"Grid references, lats and longs, detailed loca...",y,NaN,Taxon is listed as ssp. gracillimum so subsp. ...,obscured,sensitive
248,4221190,Asplenium bulbiferum,Hen And Chickens,Asplenium bulbiferum,https://id.biodiversity.org.au/node/apni/2887052,dr884,"[{'key': 'Grouping', 'value': 'Ferns & Allies'...",Ferns & Allies,P,NaN,NaN,"Grid references, lats and longs, detailed loca...",y,NaN,All species are listed as either Endangered or...,obscured,sensitive
249,4221191,Lagenophora sublyrata,Slender Bottle-daisy,Lagenophora sublyrata,https://id.biodiversity.org.au/name/apni/51338482,dr884,"[{'key': 'Grouping', 'value': 'Garden Trade -O...",Garden Trade -Other Ornamental Horticulture pl...,P,NaN,NaN,"Grid references, lats and longs, detailed loca...",y,NaN,All species are listed as either Endangered or...,obscured,sensitive


In [339]:
conservationlist = pd.read_csv(sourcedir + "sa-ala-conservation.csv", dtype=str)
conservationlist['co_status'] = conservationlist['status']
conservationlist['sa_geoprivacy'] = 'open'
conservationlist

,id,name,commonName,scientificName,lsid,dataResourceUid,kvpValues,class,order,family,...,scientificNameAuthorship,status,npwActStatusComments,epbcStatus,epbcActStatusComments,subspeciesAuthor,majorGroup,subSpecies,co_status,sa_geoprivacy
0,4032241,Ornithorhynchus anatinus,Platypus,Ornithorhynchus anatinus,https://biodiversity.org.au/afd/taxa/ac61fd14-...,dr653,"[{'key': 'class', 'value': 'MAMMALIA'}, {'key'...",MAMMALIA,MONOTREMATA,ORNITHORHYNCHIDAE,...,"(Shaw, 1799)",Endangered,NaN,NaN,NaN,NaN,NaN,NaN,Endangered,open
1,4032333,Tachyglossus aculeatus multiaculeatus,Short-beaked Echidna (kangaroo Island),Tachyglossus aculeatus multiaculeatus,https://biodiversity.org.au/afd/taxa/e9289876-...,dr653,"[{'key': 'class', 'value': 'MAMMALIA'}, {'key'...",MAMMALIA,MONOTREMATA,TACHYGLOSSIDAE,...,"(Rothschild, 1905)",Endangered,Listing official as off 1/Jan/2020.,EN,NaN,NaN,NaN,NaN,Endangered,open
2,4031684,Myrmecobius fasciatus,Numbat,Myrmecobius fasciatus,https://biodiversity.org.au/afd/taxa/6c72d199-...,dr653,"[{'key': 'class', 'value': 'MAMMALIA'}, {'key'...",MAMMALIA,DASYUROMORPHIA,MYRMECOBIIDAE,...,"Waterhouse, 1836",Endangered,Introduced,EN,Introduced to South Australia. Changed from VU...,NaN,NaN,NaN,Endangered,open
3,4032310,Dasycercus blythi,Brush-tailed Mulgara,Dasycercus blythi,https://biodiversity.org.au/afd/taxa/926b1c33-...,dr653,"[{'key': 'class', 'value': 'MAMMALIA'}, {'key'...",MAMMALIA,DASYUROMORPHIA,DASYURIDAE,...,"(Krefft, 1867)",Endangered,NaN,NaN,NaN,NaN,NaN,NaN,Endangered,open
4,4031916,Dasyuroides byrnei,Kowari,Dasyuroides byrnei,https://biodiversity.org.au/afd/taxa/c342ff42-...,dr653,"[{'key': 'class', 'value': 'MAMMALIA'}, {'key'...",MAMMALIA,DASYUROMORPHIA,DASYURIDAE,...,"(Spencer, 1896)",Endangered,Changed from V to E 1/Jan/2020.,VU,NaN,NaN,NaN,NaN,Endangered,open
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167,4032775,Thelymitra matthewsii,Spiral Sun-orchid,Thelymitra matthewsii,https://id.biodiversity.org.au/taxon/apni/5141...,dr653,"[{'key': 'family', 'value': 'ORCHIDACEAE'}, {'...",NaN,NaN,ORCHIDACEAE,...,Thelymitra matthewsii Cheeseman,Endangered,NaN,VU,2000-07-16 00:00:00,NaN,Angiospermae,NaN,Endangered,open
1168,4032248,Thelymitra occidentalis,Western Azure Sun Orchid,Thelymitra occidentalis,https://id.biodiversity.org.au/taxon/apni/5141...,dr653,"[{'key': 'family', 'value': 'ORCHIDACEAE'}, {'...",NaN,NaN,ORCHIDACEAE,...,Thelymitra occidentalis Jeanes,Rare,RA d(i),NaN,NaN,NaN,Angiospermae,NaN,Rare,open
1169,4031900,Thelymitra orientalis,Plum Sun-orchid,Thelymitra orientalis,https://id.biodiversity.org.au/taxon/apni/5141...,dr653,"[{'key': 'family', 'value': 'ORCHIDACEAE'}, {'...",NaN,NaN,ORCHIDACEAE,...,Thelymitra orientalis R.J.Bates,Rare,NaN,NaN,NaN,NaN,Angiospermae,NaN,Rare,open
1170,4032428,Thelymitra peniculata,Blue Star Sun-orchid,Thelymitra peniculata,https://id.biodiversity.org.au/taxon/apni/5141...,dr653,"[{'key': 'family', 'value': 'ORCHIDACEAE'}, {'...",NaN,NaN,ORCHIDACEAE,...,Thelymitra peniculata Jeanes,Vulnerable,Warning: Conservation status data auto-copied ...,NaN,NaN,NaN,Angiospermae,NaN,Vulnerable,open


In [340]:
# Sensitive list does not have a status.
# Records need status from Conservation list for those species
slist = sensitivelist['name']
slistloc = pd.DataFrame(conservationlist.loc[conservationlist['name'].isin(slist)])
# dfsistloc = pd.DataFrame(slistloc)
slistloc['sa_geoprivacy'] = 'obscured'  # species in both sensitivelist and conservationlist so get sensitive list geoprivacy
slistloc

,id,name,commonName,scientificName,lsid,dataResourceUid,kvpValues,class,order,family,...,scientificNameAuthorship,status,npwActStatusComments,epbcStatus,epbcActStatusComments,subspeciesAuthor,majorGroup,subSpecies,co_status,sa_geoprivacy
130,4032000,Elanus scriptus,Letter-winged Kite,Elanus scriptus,https://biodiversity.org.au/afd/taxa/4a6524fb-...,dr653,"[{'key': 'class', 'value': 'AVES'}, {'key': 'o...",AVES,ACCIPITRIFORMES,ACCIPITRIDAE,...,"Gould, 1842",Vulnerable,Changed from R to V 1/Jan/2020.,NaN,NaN,NaN,NaN,NaN,Vulnerable,obscured
131,4032490,Haliaeetus leucogaster,White-bellied Sea-eagle,Haliaeetus (Pontoaetus) leucogaster,https://biodiversity.org.au/afd/taxa/dc420306-...,dr653,"[{'key': 'class', 'value': 'AVES'}, {'key': 'o...",AVES,ACCIPITRIFORMES,ACCIPITRIDAE,...,"(J.F. Gmelin, 1788)",Endangered,NaN,NaN,NaN,NaN,NaN,NaN,Endangered,obscured
134,4032081,Lophoictinia isura,Square-tailed Kite,Lophoictinia isura,https://biodiversity.org.au/afd/taxa/715a2874-...,dr653,"[{'key': 'class', 'value': 'AVES'}, {'key': 'o...",AVES,ACCIPITRIFORMES,ACCIPITRIDAE,...,"(Gould, 1838)",Endangered,NaN,NaN,NaN,NaN,NaN,NaN,Endangered,obscured
148,4032089,Thinornis cucullatus cucullatus,Eastern Hooded Dotteral,Thinornis cucullatus cucullatus,https://biodiversity.org.au/afd/taxa/a1591e76-...,dr653,"[{'key': 'class', 'value': 'AVES'}, {'key': 'o...",AVES,CHARADRIIFORMES,CHARADRIIDAE,...,"(J.F. Gmelin, 1789)",Vulnerable,NaN,VU,listed 6/11/2014,NaN,NaN,NaN,Vulnerable,obscured
181,4032696,Falco hypoleucos,Grey Falcon,Falco (Hierofalco) hypoleucos,https://biodiversity.org.au/afd/taxa/4c73a934-...,dr653,"[{'key': 'class', 'value': 'AVES'}, {'key': 'o...",AVES,FALCONIFORMES,FALCONIDAE,...,"Gould, 1841",Rare,NaN,VU,listed July 2020,NaN,NaN,NaN,Rare,obscured
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163,4032315,Thelymitra inflata,Inflated Sun-orchid,Thelymitra inflata,https://id.biodiversity.org.au/taxon/apni/5141...,dr653,"[{'key': 'family', 'value': 'ORCHIDACEAE'}, {'...",NaN,NaN,ORCHIDACEAE,...,Thelymitra inflata Jeanes,Vulnerable,"VU A2a;C2a(i), updated null to V, Apr 2007",NaN,NaN,NaN,Angiospermae,NaN,Vulnerable,obscured
1165,4032765,Thelymitra latifolia,Blue Star Sun-orchid,Thelymitra latifolia,https://id.biodiversity.org.au/taxon/apni/5141...,dr653,"[{'key': 'family', 'value': 'ORCHIDACEAE'}, {'...",NaN,NaN,ORCHIDACEAE,...,Thelymitra latifolia R.J.Bates,Vulnerable,Warning: Conservation status data auto-copied ...,NaN,NaN,NaN,Angiospermae,NaN,Vulnerable,obscured
1166,4031963,Thelymitra malvina,Mauvetuft Sun-orchid,Thelymitra malvina,https://id.biodiversity.org.au/taxon/apni/5141...,dr653,"[{'key': 'family', 'value': 'ORCHIDACEAE'}, {'...",NaN,NaN,ORCHIDACEAE,...,"Thelymitra malvina M.A.Clem., D.L.Jones & Molloy",Endangered,NaN,NaN,NaN,NaN,Angiospermae,NaN,Endangered,obscured
1167,4032775,Thelymitra matthewsii,Spiral Sun-orchid,Thelymitra matthewsii,https://id.biodiversity.org.au/taxon/apni/5141...,dr653,"[{'key': 'family', 'value': 'ORCHIDACEAE'}, {'...",NaN,NaN,ORCHIDACEAE,...,Thelymitra matthewsii Cheeseman,Endangered,NaN,VU,2000-07-16 00:00:00,NaN,Angiospermae,NaN,Endangered,obscured


In [341]:
mlist=  sensitivelist.merge(slistloc[['status','name', 'sa_geoprivacy']],how="left",left_on="name",right_on="name")
mlist.sa_geoprivacy_y = mlist.sa_geoprivacy_y.fillna(value='obscured')
mlist

,id,name,commonName,scientificName,lsid,dataResourceUid,kvpValues,Grouping,SPECIESTYPE,taxonID,vernacular name,Clarifier,Threaten a species/environment;,Threaten someones privacy;,Reason,sa_geoprivacy_x,status_x,status_y,sa_geoprivacy_y
0,24233,Tiliqua adelaidensis,Pygmy Bluetongue,Tiliqua adelaidensis,https://biodiversity.org.au/afd/taxa/a70c19ab-...,dr884,"[{'key': 'Grouping', 'value': 'Reptiles'}, {'k...",Reptiles,R,Y02572,Pygmy Blue-tongue Lizard (listed as Pygmy Blue...,All known location and burrow records,y,y,The Pygmy Blue-tongue Lizard is endemic to SA ...,obscured,sensitive,NaN,obscured
1,24377,Calyptorhynchus banksii graptogyne,Red-tailed Black Cockatoo (south-east),Calyptorhynchus (Calyptorhynchus) banksii grap...,https://biodiversity.org.au/afd/taxa/f8ecc7e3-...,dr884,"[{'key': 'Grouping', 'value': 'Cockatoos'}, {'...",Cockatoos,B,Y09312,Red-tailed Black-Cockatoo (south east subspecies),All known nest hollows and artificial nest boxes.,y,NaN,These Cockatoo species are listed under the EP...,obscured,sensitive,Endangered,obscured
2,24336,Calyptorhynchus lathami halmaturinus,Glossy Black-cockatoo (kangaroo Island Subspec...,Calyptorhynchus (Calyptorhynchus) lathami halm...,https://biodiversity.org.au/afd/taxa/d2f5121c-...,dr884,"[{'key': 'Grouping', 'value': 'Cockatoos'}, {'...",Cockatoos,B,C00265,Glossy Black-Cockatoo (Kangaroo Island subspec...,All known nest hollows and artificial nest boxes.,y,NaN,These Cockatoo species are listed under the EP...,obscured,sensitive,NaN,obscured
3,24331,Lophochroa leadbeateri,Major Mitchell's Cockatoo,Lophochroa leadbeateri,https://biodiversity.org.au/afd/taxa/5815e99d-...,dr884,"[{'key': 'Grouping', 'value': 'Cockatoos'}, {'...",Cockatoos,B,U00270,Major Mitchell's Cockatoo (listed as Cacatua l...,All known nest hollows and artificial nest boxes.,y,NaN,These Cockatoo species are listed under the EP...,obscured,sensitive,NaN,obscured
4,24267,Calyptorhynchus funereus,Yellow-tailed Black-cockatoo,Zanda funerea,https://biodiversity.org.au/afd/taxa/145b081d-...,dr884,"[{'key': 'Grouping', 'value': 'Cockatoos'}, {'...",Cockatoos,B,G00267,Yellow-tailed Black Cockatoo,All known nest hollows and artificial nest boxes.,y,NaN,some associated subspecies records are sensitive,obscured,sensitive,NaN,obscured
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,4221188,Ixodia achillaeoides subsp. arenicola,NaN,Ixodia achillaeoides subsp. arenicola,https://id.biodiversity.org.au/node/apni/2916083,dr884,"[{'key': 'Grouping', 'value': 'Garden Trade -O...",Garden Trade -Other Ornamental Horticulture pl...,P,ALA concept,NaN,"Grid references, lats and longs, detailed loca...",y,NaN,All species are listed as either Endangered or...,obscured,sensitive,NaN,obscured
247,4221189,Asplenium bulbiferum subsp. gracillimum,Mother Spleenwort,Asplenium bulbiferum subsp. gracillimum,https://id.biodiversity.org.au/node/apni/2910577,dr884,"[{'key': 'Grouping', 'value': 'Ferns & Allies'...",Ferns & Allies,P,ALA concept,NaN,"Grid references, lats and longs, detailed loca...",y,NaN,Taxon is listed as ssp. gracillimum so subsp. ...,obscured,sensitive,NaN,obscured
248,4221190,Asplenium bulbiferum,Hen And Chickens,Asplenium bulbiferum,https://id.biodiversity.org.au/node/apni/2887052,dr884,"[{'key': 'Grouping', 'value': 'Ferns & Allies'...",Ferns & Allies,P,NaN,NaN,"Grid references, lats and longs, detailed loca...",y,NaN,All species are listed as either Endangered or...,obscured,sensitive,NaN,obscured
249,4221191,Lagenophora sublyrata,Slender Bottle-daisy,Lagenophora sublyrata,https://id.biodiversity.org.au/name/apni/51338482,dr884,"[{'key': 'Grouping', 'value': 'Garden Trade -O...",Garden Trade -Other Ornamental Horticulture pl...,P,NaN,NaN,"Grid references, lats and longs, detailed loca...",y,NaN,All species are listed as either Endangered or...,obscured,sensitive,Vulnerable,obscured


In [342]:
sensitivelist['status']  = mlist.status_y.fillna(mlist.status_x)
sensitivelist['sa_geoprivacy'] = mlist.sa_geoprivacy_y
sensitivelist

,id,name,commonName,scientificName,lsid,dataResourceUid,kvpValues,Grouping,SPECIESTYPE,taxonID,vernacular name,Clarifier,Threaten a species/environment;,Threaten someones privacy;,Reason,sa_geoprivacy,status
0,24233,Tiliqua adelaidensis,Pygmy Bluetongue,Tiliqua adelaidensis,https://biodiversity.org.au/afd/taxa/a70c19ab-...,dr884,"[{'key': 'Grouping', 'value': 'Reptiles'}, {'k...",Reptiles,R,Y02572,Pygmy Blue-tongue Lizard (listed as Pygmy Blue...,All known location and burrow records,y,y,The Pygmy Blue-tongue Lizard is endemic to SA ...,obscured,sensitive
1,24377,Calyptorhynchus banksii graptogyne,Red-tailed Black Cockatoo (south-east),Calyptorhynchus (Calyptorhynchus) banksii grap...,https://biodiversity.org.au/afd/taxa/f8ecc7e3-...,dr884,"[{'key': 'Grouping', 'value': 'Cockatoos'}, {'...",Cockatoos,B,Y09312,Red-tailed Black-Cockatoo (south east subspecies),All known nest hollows and artificial nest boxes.,y,NaN,These Cockatoo species are listed under the EP...,obscured,Endangered
2,24336,Calyptorhynchus lathami halmaturinus,Glossy Black-cockatoo (kangaroo Island Subspec...,Calyptorhynchus (Calyptorhynchus) lathami halm...,https://biodiversity.org.au/afd/taxa/d2f5121c-...,dr884,"[{'key': 'Grouping', 'value': 'Cockatoos'}, {'...",Cockatoos,B,C00265,Glossy Black-Cockatoo (Kangaroo Island subspec...,All known nest hollows and artificial nest boxes.,y,NaN,These Cockatoo species are listed under the EP...,obscured,sensitive
3,24331,Lophochroa leadbeateri,Major Mitchell's Cockatoo,Lophochroa leadbeateri,https://biodiversity.org.au/afd/taxa/5815e99d-...,dr884,"[{'key': 'Grouping', 'value': 'Cockatoos'}, {'...",Cockatoos,B,U00270,Major Mitchell's Cockatoo (listed as Cacatua l...,All known nest hollows and artificial nest boxes.,y,NaN,These Cockatoo species are listed under the EP...,obscured,sensitive
4,24267,Calyptorhynchus funereus,Yellow-tailed Black-cockatoo,Zanda funerea,https://biodiversity.org.au/afd/taxa/145b081d-...,dr884,"[{'key': 'Grouping', 'value': 'Cockatoos'}, {'...",Cockatoos,B,G00267,Yellow-tailed Black Cockatoo,All known nest hollows and artificial nest boxes.,y,NaN,some associated subspecies records are sensitive,obscured,sensitive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,4221188,Ixodia achillaeoides subsp. arenicola,NaN,Ixodia achillaeoides subsp. arenicola,https://id.biodiversity.org.au/node/apni/2916083,dr884,"[{'key': 'Grouping', 'value': 'Garden Trade -O...",Garden Trade -Other Ornamental Horticulture pl...,P,ALA concept,NaN,"Grid references, lats and longs, detailed loca...",y,NaN,All species are listed as either Endangered or...,obscured,sensitive
247,4221189,Asplenium bulbiferum subsp. gracillimum,Mother Spleenwort,Asplenium bulbiferum subsp. gracillimum,https://id.biodiversity.org.au/node/apni/2910577,dr884,"[{'key': 'Grouping', 'value': 'Ferns & Allies'...",Ferns & Allies,P,ALA concept,NaN,"Grid references, lats and longs, detailed loca...",y,NaN,Taxon is listed as ssp. gracillimum so subsp. ...,obscured,sensitive
248,4221190,Asplenium bulbiferum,Hen And Chickens,Asplenium bulbiferum,https://id.biodiversity.org.au/node/apni/2887052,dr884,"[{'key': 'Grouping', 'value': 'Ferns & Allies'...",Ferns & Allies,P,NaN,NaN,"Grid references, lats and longs, detailed loca...",y,NaN,All species are listed as either Endangered or...,obscured,sensitive
249,4221191,Lagenophora sublyrata,Slender Bottle-daisy,Lagenophora sublyrata,https://id.biodiversity.org.au/name/apni/51338482,dr884,"[{'key': 'Grouping', 'value': 'Garden Trade -O...",Garden Trade -Other Ornamental Horticulture pl...,P,NaN,NaN,"Grid references, lats and longs, detailed loca...",y,NaN,All species are listed as either Endangered or...,obscured,Vulnerable


In [353]:
# join them in a way that works for inat (eg sensitive list, geoprivacy = 'obscured'
statelist = conservationlist[['name','lsid','status', 'sa_geoprivacy']].merge(sensitivelist[['name', 'sa_geoprivacy']], how="left",on='name')
# sa_geoprivacy, if value is NaN in sa_geoprivacy_y it means the value is in the conservation list only so set value to 'open'
statelist['sa_geoprivacy_y'] = statelist['sa_geoprivacy_y'].fillna('open')  # geoprivacy upate conservationlist that are not in sensitivelist
statelist['sa_geoprivacy'] = statelist['sa_geoprivacy_y']
statelist

,name,lsid,status,sa_geoprivacy_x,sa_geoprivacy_y,sa_geoprivacy
0,Ornithorhynchus anatinus,https://biodiversity.org.au/afd/taxa/ac61fd14-...,Endangered,open,open,open
1,Tachyglossus aculeatus multiaculeatus,https://biodiversity.org.au/afd/taxa/e9289876-...,Endangered,open,open,open
2,Myrmecobius fasciatus,https://biodiversity.org.au/afd/taxa/6c72d199-...,Endangered,open,open,open
3,Dasycercus blythi,https://biodiversity.org.au/afd/taxa/926b1c33-...,Endangered,open,open,open
4,Dasyuroides byrnei,https://biodiversity.org.au/afd/taxa/c342ff42-...,Endangered,open,open,open
...,...,...,...,...,...,...
1170,Thelymitra matthewsii,https://id.biodiversity.org.au/taxon/apni/5141...,Endangered,open,obscured,obscured
1171,Thelymitra occidentalis,https://id.biodiversity.org.au/taxon/apni/5141...,Rare,open,open,open
1172,Thelymitra orientalis,https://id.biodiversity.org.au/taxon/apni/5141...,Rare,open,open,open
1173,Thelymitra peniculata,https://id.biodiversity.org.au/taxon/apni/5141...,Vulnerable,open,obscured,obscured


In [354]:
# join them in a way that works for inat (eg sensitive list, geoprivacy = 'obscured'
# statelist = pd.concat([sensitivelist[[ 'taxonID', 'name',  'status', 'sa_geoprivacy', 'lsid']],
#                        conservationlist[[ 'taxonID', 'name',  'status', 'sa_geoprivacy', 'lsid']]]).drop_duplicates()
# retrieve binomial and trinomial names from GBIF
parsednames = lf.gbifparse(statelist)
# parsednames = pd.read_csv(sourcedir + "act-gbif.csv")
statelist = statelist.merge(parsednames,how="left",left_on="name",right_on="scientificName")
statelist = statelist.drop_duplicates()
numfullstatelist = len(statelist.index)
statelist['scientificName'] = statelist['canonicalName']
statelist

,name,lsid,status,sa_geoprivacy_x,sa_geoprivacy_y,sa_geoprivacy,scientificName,type,genusOrAbove,specificEpithet,...,canonicalName,canonicalNameComplete,canonicalNameWithMarker,rankMarker,infraSpecificEpithet,infraGeneric,strain,notho,bracketAuthorship,bracketYear
0,Ornithorhynchus anatinus,https://biodiversity.org.au/afd/taxa/ac61fd14-...,Endangered,open,open,open,Ornithorhynchus anatinus,SCIENTIFIC,Ornithorhynchus,anatinus,...,Ornithorhynchus anatinus,Ornithorhynchus anatinus,Ornithorhynchus anatinus,sp.,NaN,NaN,NaN,NaN,NaN,NaN
1,Tachyglossus aculeatus multiaculeatus,https://biodiversity.org.au/afd/taxa/e9289876-...,Endangered,open,open,open,Tachyglossus aculeatus multiaculeatus,SCIENTIFIC,Tachyglossus,aculeatus,...,Tachyglossus aculeatus multiaculeatus,Tachyglossus aculeatus multiaculeatus,Tachyglossus aculeatus multiaculeatus,infrasp.,multiaculeatus,NaN,NaN,NaN,NaN,NaN
2,Myrmecobius fasciatus,https://biodiversity.org.au/afd/taxa/6c72d199-...,Endangered,open,open,open,Myrmecobius fasciatus,SCIENTIFIC,Myrmecobius,fasciatus,...,Myrmecobius fasciatus,Myrmecobius fasciatus,Myrmecobius fasciatus,sp.,NaN,NaN,NaN,NaN,NaN,NaN
3,Dasycercus blythi,https://biodiversity.org.au/afd/taxa/926b1c33-...,Endangered,open,open,open,Dasycercus blythi,SCIENTIFIC,Dasycercus,blythi,...,Dasycercus blythi,Dasycercus blythi,Dasycercus blythi,sp.,NaN,NaN,NaN,NaN,NaN,NaN
4,Dasyuroides byrnei,https://biodiversity.org.au/afd/taxa/c342ff42-...,Endangered,open,open,open,Dasyuroides byrnei,SCIENTIFIC,Dasyuroides,byrnei,...,Dasyuroides byrnei,Dasyuroides byrnei,Dasyuroides byrnei,sp.,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1176,Thelymitra matthewsii,https://id.biodiversity.org.au/taxon/apni/5141...,Endangered,open,obscured,obscured,Thelymitra matthewsii,SCIENTIFIC,Thelymitra,matthewsii,...,Thelymitra matthewsii,Thelymitra matthewsii,Thelymitra matthewsii,sp.,NaN,NaN,NaN,NaN,NaN,NaN
1177,Thelymitra occidentalis,https://id.biodiversity.org.au/taxon/apni/5141...,Rare,open,open,open,Thelymitra occidentalis,SCIENTIFIC,Thelymitra,occidentalis,...,Thelymitra occidentalis,Thelymitra occidentalis,Thelymitra occidentalis,sp.,NaN,NaN,NaN,NaN,NaN,NaN
1178,Thelymitra orientalis,https://id.biodiversity.org.au/taxon/apni/5141...,Rare,open,open,open,Thelymitra orientalis,SCIENTIFIC,Thelymitra,orientalis,...,Thelymitra orientalis,Thelymitra orientalis,Thelymitra orientalis,sp.,NaN,NaN,NaN,NaN,NaN,NaN
1179,Thelymitra peniculata,https://id.biodiversity.org.au/taxon/apni/5141...,Vulnerable,open,obscured,obscured,Thelymitra peniculata,SCIENTIFIC,Thelymitra,peniculata,...,Thelymitra peniculata,Thelymitra peniculata,Thelymitra peniculata,sp.,NaN,NaN,NaN,NaN,NaN,NaN


In [355]:
# Identify records that won't comply with iNaturalist species names
noncomply = statelist[statelist['type'].isin(['INFORMAL','CULTIVAR','HYBRID', 'BLACKLISTED']) ]
noncomply

,name,lsid,status,sa_geoprivacy_x,sa_geoprivacy_y,sa_geoprivacy,scientificName,type,genusOrAbove,specificEpithet,...,canonicalName,canonicalNameComplete,canonicalNameWithMarker,rankMarker,infraSpecificEpithet,infraGeneric,strain,notho,bracketAuthorship,bracketYear
372,Grevillea angustiloba ssp.,ALA_DR653_364,Endangered,open,open,open,Grevillea angustiloba subsp.,INFORMAL,Grevillea,angustiloba,...,Grevillea angustiloba subsp.,Grevillea angustiloba subsp.,Grevillea angustiloba subsp.,subsp.,NaN,NaN,NaN,NaN,NaN,NaN
480,Billardiera sp. Yorke Peninsula (P.C.Heyligers...,https://id.biodiversity.org.au/node/apni/2919736,Endangered,open,open,open,Billardiera spec.,INFORMAL,Billardiera,NaN,...,Billardiera spec.,Billardiera spec. Yorke Peninsula,Billardiera spec. Yorke Peninsula,sp.,NaN,NaN,Yorke Peninsula,NaN,NaN,NaN
567,Erodium sp. Blesing (J.R.Wheeler 471),https://id.biodiversity.org.au/node/apni/2912581,Rare,open,open,open,Erodium spec.,INFORMAL,Erodium,NaN,...,Erodium spec.,Erodium spec. Blesing,Erodium spec. Blesing,sp.,NaN,NaN,Blesing,NaN,NaN,NaN
620,Abutilon oxycarpum ssp. Prostrate (A.A.Mitchel...,https://id.biodiversity.org.au/node/apni/2906856,Rare,open,open,open,Abutilon oxycarpum subsp.,INFORMAL,Abutilon,oxycarpum,...,Abutilon oxycarpum subsp.,Abutilon oxycarpum subsp. Prostrate,Abutilon oxycarpum subsp. Prostrate,subsp.,NaN,NaN,Prostrate,NaN,NaN,NaN
623,Sida sp. Wakaya Desert (C.Dunlop 1984),https://id.biodiversity.org.au/node/apni/2889258,Rare,open,open,open,Sida spec.,INFORMAL,Sida,NaN,...,Sida spec.,Sida spec. Wakaya Desert,Sida spec. Wakaya Desert,sp.,NaN,NaN,Wakaya Desert,NaN,NaN,NaN
715,Asperula sp. A (A.B.Cashmore September 1933),https://id.biodiversity.org.au/node/apni/2903743,Endangered,open,open,open,Asperula spec.,INFORMAL,Asperula,NaN,...,Asperula spec.,Asperula spec. A,Asperula spec. A,sp.,NaN,NaN,A,NaN,NaN,NaN
752,Glossostigma sp. Long stout-pedicelled (W.R.Ba...,https://id.biodiversity.org.au/node/apni/8016018,Vulnerable,open,open,open,Glossostigma stout-pedicelled sp.Long,INFORMAL,Glossostigma,stout-pedicelled,...,Glossostigma stout-pedicelled sp.Long,Glossostigma stout-pedicelled sp. sp.Long (W.R...,Glossostigma stout-pedicelled sp. sp.Long,sp.,sp.Long,NaN,NaN,NaN,W.R.Barker,2481.0
758,Stemodia sp. Haegii (J.Z.Weber 9055),https://id.biodiversity.org.au/node/apni/2914024,Rare,open,open,open,Stemodia spec.,INFORMAL,Stemodia,NaN,...,Stemodia spec.,Stemodia spec. Haegii,Stemodia spec. Haegii,sp.,NaN,NaN,Haegii,NaN,NaN,NaN
783,Plantago sp. A (A.C.Robinson 704),https://id.biodiversity.org.au/node/apni/2907792,Rare,open,open,open,Plantago spec.,INFORMAL,Plantago,NaN,...,Plantago spec.,Plantago spec. A,Plantago spec. A,sp.,NaN,NaN,A,NaN,NaN,NaN
916,Thysanotus tuberosus ssp.,ALA_DR653_908,Vulnerable,open,obscured,obscured,Thysanotus tuberosus subsp.,INFORMAL,Thysanotus,tuberosus,...,Thysanotus tuberosus subsp.,Thysanotus tuberosus subsp.,Thysanotus tuberosus subsp.,subsp.,NaN,NaN,NaN,NaN,NaN,NaN


In [356]:
# remove records that do not comply
statelist = statelist[~statelist['type'].isin(['INFORMAL','CULTIVAR','HYBRID', 'BLACKLISTED']) ]
# statelist = pd.DataFrame(statelist[['sa_scientificName','sa_status','sa_geoprivacy','lsid']]).drop_duplicates()
statelist.to_csv(sourcedir + "statelist.csv", index=False )
statelist

,name,lsid,status,sa_geoprivacy_x,sa_geoprivacy_y,sa_geoprivacy,scientificName,type,genusOrAbove,specificEpithet,...,canonicalName,canonicalNameComplete,canonicalNameWithMarker,rankMarker,infraSpecificEpithet,infraGeneric,strain,notho,bracketAuthorship,bracketYear
0,Ornithorhynchus anatinus,https://biodiversity.org.au/afd/taxa/ac61fd14-...,Endangered,open,open,open,Ornithorhynchus anatinus,SCIENTIFIC,Ornithorhynchus,anatinus,...,Ornithorhynchus anatinus,Ornithorhynchus anatinus,Ornithorhynchus anatinus,sp.,NaN,NaN,NaN,NaN,NaN,NaN
1,Tachyglossus aculeatus multiaculeatus,https://biodiversity.org.au/afd/taxa/e9289876-...,Endangered,open,open,open,Tachyglossus aculeatus multiaculeatus,SCIENTIFIC,Tachyglossus,aculeatus,...,Tachyglossus aculeatus multiaculeatus,Tachyglossus aculeatus multiaculeatus,Tachyglossus aculeatus multiaculeatus,infrasp.,multiaculeatus,NaN,NaN,NaN,NaN,NaN
2,Myrmecobius fasciatus,https://biodiversity.org.au/afd/taxa/6c72d199-...,Endangered,open,open,open,Myrmecobius fasciatus,SCIENTIFIC,Myrmecobius,fasciatus,...,Myrmecobius fasciatus,Myrmecobius fasciatus,Myrmecobius fasciatus,sp.,NaN,NaN,NaN,NaN,NaN,NaN
3,Dasycercus blythi,https://biodiversity.org.au/afd/taxa/926b1c33-...,Endangered,open,open,open,Dasycercus blythi,SCIENTIFIC,Dasycercus,blythi,...,Dasycercus blythi,Dasycercus blythi,Dasycercus blythi,sp.,NaN,NaN,NaN,NaN,NaN,NaN
4,Dasyuroides byrnei,https://biodiversity.org.au/afd/taxa/c342ff42-...,Endangered,open,open,open,Dasyuroides byrnei,SCIENTIFIC,Dasyuroides,byrnei,...,Dasyuroides byrnei,Dasyuroides byrnei,Dasyuroides byrnei,sp.,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1176,Thelymitra matthewsii,https://id.biodiversity.org.au/taxon/apni/5141...,Endangered,open,obscured,obscured,Thelymitra matthewsii,SCIENTIFIC,Thelymitra,matthewsii,...,Thelymitra matthewsii,Thelymitra matthewsii,Thelymitra matthewsii,sp.,NaN,NaN,NaN,NaN,NaN,NaN
1177,Thelymitra occidentalis,https://id.biodiversity.org.au/taxon/apni/5141...,Rare,open,open,open,Thelymitra occidentalis,SCIENTIFIC,Thelymitra,occidentalis,...,Thelymitra occidentalis,Thelymitra occidentalis,Thelymitra occidentalis,sp.,NaN,NaN,NaN,NaN,NaN,NaN
1178,Thelymitra orientalis,https://id.biodiversity.org.au/taxon/apni/5141...,Rare,open,open,open,Thelymitra orientalis,SCIENTIFIC,Thelymitra,orientalis,...,Thelymitra orientalis,Thelymitra orientalis,Thelymitra orientalis,sp.,NaN,NaN,NaN,NaN,NaN,NaN
1179,Thelymitra peniculata,https://id.biodiversity.org.au/taxon/apni/5141...,Vulnerable,open,obscured,obscured,Thelymitra peniculata,SCIENTIFIC,Thelymitra,peniculata,...,Thelymitra peniculata,Thelymitra peniculata,Thelymitra peniculata,sp.,NaN,NaN,NaN,NaN,NaN,NaN


In [357]:
parsednames['type'].unique()

array(['SCIENTIFIC', 'INFORMAL'], dtype=object)

In [358]:
dupinformation = statelist.groupby('scientificName').filter(lambda x: len(x) > 1)#.sort('size',ascending=False)
dupinformation


,name,lsid,status,sa_geoprivacy_x,sa_geoprivacy_y,sa_geoprivacy,scientificName,type,genusOrAbove,specificEpithet,...,canonicalName,canonicalNameComplete,canonicalNameWithMarker,rankMarker,infraSpecificEpithet,infraGeneric,strain,notho,bracketAuthorship,bracketYear


### 4. Equivalent IUCN statuses

In [359]:
# check for duplicates with conflicting information
iucn_statuses = {'Not Evaluated', 'Data Deficient', 'Least Concern', 'Near Threatened', 'Vulnerable', 'Endangered', 'Critically Endangered', 'Extinct in the Wild','Extinct'}
statelist.groupby(['status'])['status'].count()

status
Endangered    299
Rare          605
Vulnerable    252
Name: status, dtype: int64

In [360]:
# these will be used to populate the iucn_equivalent field
iucnStatusMappings = {
    'rare': 'Vulnerable',
    'endangered': 'Endangered',
    'vulnerable':'Vulnerable'
}

### 5. Determine best place ID to use

In [361]:
inatstatuses.groupby(['place_id','place_name','place_display_name'])['place_id'].count()
# looks like 6899 - note for extract


place_id  place_name                      place_display_name            
124968    South Australia, marine waters  South Australia, marine waters      2
6899      South Australia                 South Australia, AU               124
Name: place_id, dtype: int64

## Merge iNaturalist statuses with State lists on scientificName

1. Match - updates, even if the statuses are the same we'll update the links and values anyway
2. No match - statuses to be added (additions)
   1.1 No match and no taxnomy - search for synonyms
   1.2 No match
3. Merge the other direction to see if there are deletes?


In [362]:
# join to see which lists already have a status in inaturalist based on scientificName
mergedstatuses = statelist[['scientificName','status','sa_geoprivacy','lsid']].merge(inatstatuses[['status_id','scientificName','taxon_id','user_id','description','iucn','authority','status','geoprivacy','place_id','place_display_name']],how="left",left_on='scientificName',right_on='scientificName',suffixes=(None,'_inat')).sort_values(['scientificName'])
mergedstatuses


,scientificName,status,sa_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,authority,status_inat,geoprivacy,place_id,place_display_name
472,Acacia alcockii,Rare,open,https://id.biodiversity.org.au/node/apni/2916861,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
473,Acacia ammobia,Rare,open,https://id.biodiversity.org.au/node/apni/2915565,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
474,Acacia araneosa,Endangered,open,https://id.biodiversity.org.au/node/apni/2919802,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
475,Acacia barattensis,Rare,open,https://id.biodiversity.org.au/node/apni/2905743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
476,Acacia carneorum,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2911484,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588,Zieria veronicea veronicea,Rare,open,https://id.biodiversity.org.au/node/apni/2911634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,Ziphius cavirostris,Rare,open,https://biodiversity.org.au/afd/taxa/ec880d6b-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
264,Zoothera lunulata,Rare,open,https://biodiversity.org.au/afd/taxa/4cb0a608-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
881,Zostera muelleri mucronata,Rare,open,ALA_DR653_891,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [364]:
# prepare the export fields, common to New template and Update template
# new statuses
# Taxon_Name,Status,Authority,IUCN_equivalent,Description,iNaturalist_Place_ID,url,Taxon_Geoprivacy,Username,taxon_id
# updates
# action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username
biesearchurl = "https://bie.ala.org.au/species/" # eg + "https://id.biodiversity.org.au/node/apni/2894366"
mergedstatuses['new_url'] =  biesearchurl + mergedstatuses['lsid']
mergedstatuses['new_description'] = "Listed as Threatened - refer to https://www.environment.sa.gov.au/topics/plants-and-animals/threatened-species-and-ecological-communities/threatened-species"
mergedstatuses['new_authority'] = "South Australia Department for Environment and Water"
mergedstatuses.rename(columns={'sa_geoprivacy':'new_geoprivacy'},inplace=True)
mergedstatuses['new_place_id'] = '6899'  # SA
mergedstatuses['new_username'] = 'peggydnew'
mergedstatuses['new_iucn_equivalent'] = mergedstatuses['status'].str.lower().str.strip().map(iucnStatusMappings).fillna('Vulnerable') # map to dictionary
mergedstatuses['new_status'] = mergedstatuses['status'].fillna('Sensitive')
mergedstatuses

,scientificName,status,new_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,authority,...,geoprivacy,place_id,place_display_name,new_url,new_description,new_authority,new_place_id,new_username,new_iucn_equivalent,new_status
472,Acacia alcockii,Rare,open,https://id.biodiversity.org.au/node/apni/2916861,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://bie.ala.org.au/species/https://id.biod...,Listed as Threatened - refer to https://www.en...,South Australia Department for Environment and...,6899,peggydnew,Vulnerable,Rare
473,Acacia ammobia,Rare,open,https://id.biodiversity.org.au/node/apni/2915565,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://bie.ala.org.au/species/https://id.biod...,Listed as Threatened - refer to https://www.en...,South Australia Department for Environment and...,6899,peggydnew,Vulnerable,Rare
474,Acacia araneosa,Endangered,open,https://id.biodiversity.org.au/node/apni/2919802,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://bie.ala.org.au/species/https://id.biod...,Listed as Threatened - refer to https://www.en...,South Australia Department for Environment and...,6899,peggydnew,Endangered,Endangered
475,Acacia barattensis,Rare,open,https://id.biodiversity.org.au/node/apni/2905743,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://bie.ala.org.au/species/https://id.biod...,Listed as Threatened - refer to https://www.en...,South Australia Department for Environment and...,6899,peggydnew,Vulnerable,Rare
476,Acacia carneorum,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2911484,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://bie.ala.org.au/species/https://id.biod...,Listed as Threatened - refer to https://www.en...,South Australia Department for Environment and...,6899,peggydnew,Vulnerable,Vulnerable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588,Zieria veronicea veronicea,Rare,open,https://id.biodiversity.org.au/node/apni/2911634,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://bie.ala.org.au/species/https://id.biod...,Listed as Threatened - refer to https://www.en...,South Australia Department for Environment and...,6899,peggydnew,Vulnerable,Rare
81,Ziphius cavirostris,Rare,open,https://biodiversity.org.au/afd/taxa/ec880d6b-...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://bie.ala.org.au/species/https://biodive...,Listed as Threatened - refer to https://www.en...,South Australia Department for Environment and...,6899,peggydnew,Vulnerable,Rare
264,Zoothera lunulata,Rare,open,https://biodiversity.org.au/afd/taxa/4cb0a608-...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://bie.ala.org.au/species/https://biodive...,Listed as Threatened - refer to https://www.en...,South Australia Department for Environment and...,6899,peggydnew,Vulnerable,Rare
881,Zostera muelleri mucronata,Rare,open,ALA_DR653_891,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://bie.ala.org.au/species/ALA_DR653_891,Listed as Threatened - refer to https://www.en...,South Australia Department for Environment and...,6899,peggydnew,Vulnerable,Rare


## Updates

In [365]:
# those that need to be updated - we found a status
mergedstatuses[mergedstatuses['status_id'].notnull()][['scientificName','status','status_id','taxon_id','status','new_geoprivacy','geoprivacy','authority','user_id']]

,scientificName,status,status_id,taxon_id,status,new_geoprivacy,geoprivacy,authority,user_id
348,Adiantum capillus-veneris,Vulnerable,152329,63916,Vulnerable,obscured,obscured,SA DEWNR,708886
349,Anogramma leptophylla,Rare,162866,180646,Rare,open,,"National Parks and Wildlife Act, 1972",702203
884,Arthropodium milleflorum,Vulnerable,152354,522233,Vulnerable,obscured,obscured,SA DEWNR,708886
351,Blechnum chambersii,Endangered,152381,323506,Endangered,obscured,obscured,SA DEWNR,708886
807,Brachyscome diversifolia,Endangered,152402,520376,Endangered,obscured,obscured,SA DEWNR,708886
...,...,...,...,...,...,...,...,...,...
896,Thysanotus nudicaulis,Endangered,152401,862784,Endangered,obscured,obscured,SA DEWNR,708886
304,Tiliqua adelaidensis,Endangered,152348,37472,Endangered,open,obscured,SA DEWNR,708886
341,Todea barbara,Endangered,152368,323949,Endangered,obscured,obscured,SA DEWNR,708886
752,Utricularia beaugleholei,Vulnerable,152394,508359,Vulnerable,obscured,obscured,SA DEWNR,708886


In [366]:
# updates - create the data frame
# action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
updates = pd.DataFrame(mergedstatuses[mergedstatuses['status_id'].notnull()])
updates.sort_values('scientificName')
updates['action'] = 'UPDATE'
#updates.loc[:,'action'] = 'UPDATE'
updates = updates[['action','scientificName','status_id','taxon_id','new_status','new_iucn_equivalent','new_authority','new_url','new_geoprivacy','new_place_id','new_username','new_description']]
updates.columns = updates.columns.str.replace("new_", "", regex=True)
updates = updates.rename(columns={'scientificName':'taxon_name',
                                  'status_id':'id'})
updates

,action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
348,UPDATE,Adiantum capillus-veneris,152329,63916,Vulnerable,Vulnerable,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,obscured,6899,peggydnew,Listed as Threatened - refer to https://www.en...
349,UPDATE,Anogramma leptophylla,162866,180646,Rare,Vulnerable,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,open,6899,peggydnew,Listed as Threatened - refer to https://www.en...
884,UPDATE,Arthropodium milleflorum,152354,522233,Vulnerable,Vulnerable,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,obscured,6899,peggydnew,Listed as Threatened - refer to https://www.en...
351,UPDATE,Blechnum chambersii,152381,323506,Endangered,Endangered,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,obscured,6899,peggydnew,Listed as Threatened - refer to https://www.en...
807,UPDATE,Brachyscome diversifolia,152402,520376,Endangered,Endangered,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,obscured,6899,peggydnew,Listed as Threatened - refer to https://www.en...
...,...,...,...,...,...,...,...,...,...,...,...,...
896,UPDATE,Thysanotus nudicaulis,152401,862784,Endangered,Endangered,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,obscured,6899,peggydnew,Listed as Threatened - refer to https://www.en...
304,UPDATE,Tiliqua adelaidensis,152348,37472,Endangered,Endangered,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://biodive...,open,6899,peggydnew,Listed as Threatened - refer to https://www.en...
341,UPDATE,Todea barbara,152368,323949,Endangered,Endangered,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,obscured,6899,peggydnew,Listed as Threatened - refer to https://www.en...
752,UPDATE,Utricularia beaugleholei,152394,508359,Vulnerable,Vulnerable,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,obscured,6899,peggydnew,Listed as Threatened - refer to https://www.en...


## No status in iNaturalist via straight scientificName match
The SA records that didn't match up to a status in iNaturalist

In [367]:
# to add: those that have no inaturalist status - 532!!
noinatstatus = mergedstatuses[mergedstatuses['status_id'].isnull()]
# try to match the taxon name to something in inaturalist
noinatstatus = noinatstatus.merge(inattaxa, how="left", left_on="scientificName",right_on="scientificName")
noinatstatus

,scientificName,status,new_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,authority,...,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,taxonRank,references
0,Acacia alcockii,Rare,open,https://id.biodiversity.org.au/node/apni/2916861,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,alcockii,NaN,2022-04-06T22:13:16Z,species,http://powo.science.kew.org/
1,Acacia ammobia,Rare,open,https://id.biodiversity.org.au/node/apni/2915565,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,ammobia,NaN,2022-04-05T00:59:23Z,species,http://powo.science.kew.org/
2,Acacia araneosa,Endangered,open,https://id.biodiversity.org.au/node/apni/2919802,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,araneosa,NaN,2022-04-06T22:13:57Z,species,http://powo.science.kew.org/
3,Acacia barattensis,Rare,open,https://id.biodiversity.org.au/node/apni/2905743,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,barattensis,NaN,2022-04-07T01:35:53Z,species,http://www.catalogueoflife.org/annual-checklis...
4,Acacia carneorum,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2911484,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,carneorum,NaN,2022-04-06T22:35:17Z,species,http://www.catalogueoflife.org/annual-checklis...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054,Zieria veronicea veronicea,Rare,open,https://id.biodiversity.org.au/node/apni/2911634,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Sapindales,Rutaceae,Zieria,veronicea,veronicea,2021-12-03T03:55:21Z,subspecies,http://www.ubio.org/browser/details.php?nameba...
1055,Ziphius cavirostris,Rare,open,https://biodiversity.org.au/afd/taxa/ec880d6b-...,NaN,NaN,NaN,NaN,NaN,NaN,...,Chordata,Mammalia,Artiodactyla,Ziphiidae,Ziphius,cavirostris,NaN,2022-05-31T13:14:36Z,species,http://www.catalogueoflife.org/annual-checklis...
1056,Zoothera lunulata,Rare,open,https://biodiversity.org.au/afd/taxa/4cb0a608-...,NaN,NaN,NaN,NaN,NaN,NaN,...,Chordata,Aves,Passeriformes,Turdidae,Zoothera,lunulata,NaN,2019-11-23T01:59:12Z,species,http://www.birdlife.org/datazone/speciesfactsh...
1057,Zostera muelleri mucronata,Rare,open,ALA_DR653_891,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [368]:
noinatstatus[noinatstatus['id'].notna()] # there's no status but there is a matching inat taxon (id is the taxon id)
# note: "Dendrobium" matches to both genus and section

,scientificName,status,new_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,authority,...,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,taxonRank,references
0,Acacia alcockii,Rare,open,https://id.biodiversity.org.au/node/apni/2916861,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,alcockii,NaN,2022-04-06T22:13:16Z,species,http://powo.science.kew.org/
1,Acacia ammobia,Rare,open,https://id.biodiversity.org.au/node/apni/2915565,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,ammobia,NaN,2022-04-05T00:59:23Z,species,http://powo.science.kew.org/
2,Acacia araneosa,Endangered,open,https://id.biodiversity.org.au/node/apni/2919802,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,araneosa,NaN,2022-04-06T22:13:57Z,species,http://powo.science.kew.org/
3,Acacia barattensis,Rare,open,https://id.biodiversity.org.au/node/apni/2905743,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,barattensis,NaN,2022-04-07T01:35:53Z,species,http://www.catalogueoflife.org/annual-checklis...
4,Acacia carneorum,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2911484,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,carneorum,NaN,2022-04-06T22:35:17Z,species,http://www.catalogueoflife.org/annual-checklis...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1052,Zapornia tabuensis,Rare,open,ALA_DR653_138,NaN,NaN,NaN,NaN,NaN,NaN,...,Chordata,Aves,Gruiformes,Rallidae,Zapornia,tabuensis,NaN,2022-04-13T19:38:40Z,species,http://www.birds.cornell.edu/clementschecklist...
1053,Zieria veronicea insularis,Rare,open,ALA_DR653_591,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Sapindales,Rutaceae,Zieria,veronicea,insularis,2020-03-30T19:18:21Z,subspecies,NaN
1054,Zieria veronicea veronicea,Rare,open,https://id.biodiversity.org.au/node/apni/2911634,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Sapindales,Rutaceae,Zieria,veronicea,veronicea,2021-12-03T03:55:21Z,subspecies,http://www.ubio.org/browser/details.php?nameba...
1055,Ziphius cavirostris,Rare,open,https://biodiversity.org.au/afd/taxa/ec880d6b-...,NaN,NaN,NaN,NaN,NaN,NaN,...,Chordata,Mammalia,Artiodactyla,Ziphiidae,Ziphius,cavirostris,NaN,2022-05-31T13:14:36Z,species,http://www.catalogueoflife.org/annual-checklis...


In [369]:
# there's no status but there is a matching inat taxon (id is the taxon id)
additions = pd.DataFrame(noinatstatus[noinatstatus['id'].notna()])
additions

,scientificName,status,new_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,authority,...,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,taxonRank,references
0,Acacia alcockii,Rare,open,https://id.biodiversity.org.au/node/apni/2916861,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,alcockii,NaN,2022-04-06T22:13:16Z,species,http://powo.science.kew.org/
1,Acacia ammobia,Rare,open,https://id.biodiversity.org.au/node/apni/2915565,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,ammobia,NaN,2022-04-05T00:59:23Z,species,http://powo.science.kew.org/
2,Acacia araneosa,Endangered,open,https://id.biodiversity.org.au/node/apni/2919802,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,araneosa,NaN,2022-04-06T22:13:57Z,species,http://powo.science.kew.org/
3,Acacia barattensis,Rare,open,https://id.biodiversity.org.au/node/apni/2905743,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,barattensis,NaN,2022-04-07T01:35:53Z,species,http://www.catalogueoflife.org/annual-checklis...
4,Acacia carneorum,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2911484,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,carneorum,NaN,2022-04-06T22:35:17Z,species,http://www.catalogueoflife.org/annual-checklis...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1052,Zapornia tabuensis,Rare,open,ALA_DR653_138,NaN,NaN,NaN,NaN,NaN,NaN,...,Chordata,Aves,Gruiformes,Rallidae,Zapornia,tabuensis,NaN,2022-04-13T19:38:40Z,species,http://www.birds.cornell.edu/clementschecklist...
1053,Zieria veronicea insularis,Rare,open,ALA_DR653_591,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Sapindales,Rutaceae,Zieria,veronicea,insularis,2020-03-30T19:18:21Z,subspecies,NaN
1054,Zieria veronicea veronicea,Rare,open,https://id.biodiversity.org.au/node/apni/2911634,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Sapindales,Rutaceae,Zieria,veronicea,veronicea,2021-12-03T03:55:21Z,subspecies,http://www.ubio.org/browser/details.php?nameba...
1055,Ziphius cavirostris,Rare,open,https://biodiversity.org.au/afd/taxa/ec880d6b-...,NaN,NaN,NaN,NaN,NaN,NaN,...,Chordata,Mammalia,Artiodactyla,Ziphiidae,Ziphius,cavirostris,NaN,2022-05-31T13:14:36Z,species,http://www.catalogueoflife.org/annual-checklis...


In [370]:
additions.sort_values(['scientificName'])
additions['action'] = 'ADD'
additions = additions[['action','scientificName','status_id','id','new_status','new_iucn_equivalent','new_authority','new_url','new_geoprivacy','new_place_id','new_username','new_description']]
additions.columns = additions.columns.str.replace("new_", "", regex=True)
additions = additions.rename(columns={'scientificName':'taxon_name',
                                      'id':'taxon_id',
                                  'status_id':'id'})
additions

,action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
0,ADD,Acacia alcockii,NaN,898591,Rare,Vulnerable,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,open,6899,peggydnew,Listed as Threatened - refer to https://www.en...
1,ADD,Acacia ammobia,NaN,898599,Rare,Vulnerable,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,open,6899,peggydnew,Listed as Threatened - refer to https://www.en...
2,ADD,Acacia araneosa,NaN,898624,Endangered,Endangered,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,open,6899,peggydnew,Listed as Threatened - refer to https://www.en...
3,ADD,Acacia barattensis,NaN,1252446,Rare,Vulnerable,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,open,6899,peggydnew,Listed as Threatened - refer to https://www.en...
4,ADD,Acacia carneorum,NaN,519207,Vulnerable,Vulnerable,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,open,6899,peggydnew,Listed as Threatened - refer to https://www.en...
...,...,...,...,...,...,...,...,...,...,...,...,...
1052,ADD,Zapornia tabuensis,NaN,508926,Rare,Vulnerable,South Australia Department for Environment and...,https://bie.ala.org.au/species/ALA_DR653_138,open,6899,peggydnew,Listed as Threatened - refer to https://www.en...
1053,ADD,Zieria veronicea insularis,NaN,1060165,Rare,Vulnerable,South Australia Department for Environment and...,https://bie.ala.org.au/species/ALA_DR653_591,open,6899,peggydnew,Listed as Threatened - refer to https://www.en...
1054,ADD,Zieria veronicea veronicea,NaN,323791,Rare,Vulnerable,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,open,6899,peggydnew,Listed as Threatened - refer to https://www.en...
1055,ADD,Ziphius cavirostris,NaN,41428,Rare,Vulnerable,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://biodive...,open,6899,peggydnew,Listed as Threatened - refer to https://www.en...


In [371]:
# manually had to remove spurious duplicates from output file
all = pd.concat([updates,additions])
all

,action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
348,UPDATE,Adiantum capillus-veneris,152329,63916,Vulnerable,Vulnerable,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,obscured,6899,peggydnew,Listed as Threatened - refer to https://www.en...
349,UPDATE,Anogramma leptophylla,162866,180646,Rare,Vulnerable,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,open,6899,peggydnew,Listed as Threatened - refer to https://www.en...
884,UPDATE,Arthropodium milleflorum,152354,522233,Vulnerable,Vulnerable,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,obscured,6899,peggydnew,Listed as Threatened - refer to https://www.en...
351,UPDATE,Blechnum chambersii,152381,323506,Endangered,Endangered,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,obscured,6899,peggydnew,Listed as Threatened - refer to https://www.en...
807,UPDATE,Brachyscome diversifolia,152402,520376,Endangered,Endangered,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,obscured,6899,peggydnew,Listed as Threatened - refer to https://www.en...
...,...,...,...,...,...,...,...,...,...,...,...,...
1052,ADD,Zapornia tabuensis,NaN,508926,Rare,Vulnerable,South Australia Department for Environment and...,https://bie.ala.org.au/species/ALA_DR653_138,open,6899,peggydnew,Listed as Threatened - refer to https://www.en...
1053,ADD,Zieria veronicea insularis,NaN,1060165,Rare,Vulnerable,South Australia Department for Environment and...,https://bie.ala.org.au/species/ALA_DR653_591,open,6899,peggydnew,Listed as Threatened - refer to https://www.en...
1054,ADD,Zieria veronicea veronicea,NaN,323791,Rare,Vulnerable,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://id.biod...,open,6899,peggydnew,Listed as Threatened - refer to https://www.en...
1055,ADD,Ziphius cavirostris,NaN,41428,Rare,Vulnerable,South Australia Department for Environment and...,https://bie.ala.org.au/species/https://biodive...,open,6899,peggydnew,Listed as Threatened - refer to https://www.en...


In [372]:
all.to_csv(sourcedir + "sa.csv", index=False )


In [373]:
# what didnt match to a taxon?
unknownToInat = noinatstatus[noinatstatus['id'].isna()]
unknownToInat

,scientificName,status,new_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,authority,...,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,taxonRank,references
10,Acacia genistifolia genistifolia,Endangered,open,https://id.biodiversity.org.au/name/apni/51406878,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,Althaea australis,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2912429,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,Amytornis merrotsyi merrotsyi,Vulnerable,open,https://biodiversity.org.au/afd/taxa/b78936b3-...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,Amytornis merrotsyi pedleri,Endangered,open,https://biodiversity.org.au/afd/taxa/d7c58777-...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,Amytornis striatus howei,Rare,obscured,ALA_DR653_208,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,Thelymitra merraniae,Endangered,open,https://id.biodiversity.org.au/taxon/apni/5141...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,Thinornis cucullatus cucullatus,Vulnerable,obscured,https://biodiversity.org.au/afd/taxa/a1591e76-...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1036,Wurmbea dioica lacunaria,Endangered,obscured,ALA_DR653_914,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1057,Zostera muelleri mucronata,Rare,open,ALA_DR653_891,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [374]:
noinatstatus[noinatstatus['id'].isna()].groupby('status').size()

status
Endangered    39
Rare          79
Vulnerable    42
dtype: int64

### are there any that need to be removed?
qld list count: 2517
qld inat statuses count: 653

updates to inat status: 570
additional inat status: 1355
qld statuses we can't find a taxon match for in iNaturalist: 606
total: 2531 (explainable via the various genus/section entries that we matched to in the taxonomy)

inat statuses left over: 653-570=83 that may need checking against the above

In [375]:
# inat statuses that aren't in added or updated
notaddedupdated = inatstatuses[~inatstatuses['taxon_id'].isin(updates['taxon_id'])]
notaddedupdated

,status_id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
147,152333,100616,708886,6899,16651,SA DEWNR,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Euastacus,bispinosus,,2022-06-06T16:26:39Z,Euastacus bispinosus,species,http://www.iucnredlist.org/apps/redlist/details,,,
89,180750,1038978,624851,6899,,"Department of Environment and Water, Governmen...",Endangered,,Known from just a few areas and near human dis...,obscured,...,Hibbertia,hirsuta,,2021-08-31T12:37:27Z,Hibbertia hirsuta,species,,,,
922,152427,116999,,6899,16651,SA DEWNR,endangered,https://lists.ala.org.au/speciesListItem/list/...,,,...,Pandion,haliaetus,,2022-06-27T03:09:49Z,Pandion haliaetus,species,http://www.birdlife.org/datazone/speciesfactsh...,,,
1193,152367,122043,708886,6899,16651,SA DEWNR,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Thysanotus,tuberosus,,2019-02-11T22:08:17Z,Thysanotus tuberosus,species,http://www.catalogueoflife.org/annual-checklis...,,,
931,165449,12647,702203,6899,,South Australia,EN,https://bie.ala.org.au/species/urn:lsid:biodiv...,,obscured,...,Epthianura,crocea,,2021-09-17T08:46:17Z,Epthianura crocea,species,http://www.birdlife.org/datazone/speciesfactsh...,,,
415,223574,1300466,3669610,6899,,"National Parks and Wildlife Act, 1972",NT,https://www.legislation.sa.gov.au/LZ/C/A/NATIO...,,,...,Hydrocotyle,glochidiata,,2021-10-31T04:06:15Z,Hydrocotyle glochidiata,species,http://www.plantsoftheworldonline.org/taxon/ur...,,,
590,261603,1359456,708886,6899,16651,SA DEWNR,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Brownseya,serpentina,,2022-02-17T04:32:12Z,Brownseya serpentina,species,,,,
891,152430,144487,708886,6899,16651,SA DEWNR,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Thinornis,cucullatus,,2020-01-11T02:12:54Z,Thinornis cucullatus,species,http://www.birds.cornell.edu/clementschecklist...,,,
2699,154407,19251,,6899,,SA DEWNR,Vulnerable,,,obscured,...,Polytelis,anthopeplus,,2022-06-11T01:21:17Z,Polytelis anthopeplus,species,http://www.birdlife.org/datazone/speciesfactsh...,,,
217,170165,20166,702203,6899,,Atlas of Living Australia,NT,https://bie.ala.org.au/species/urn:lsid:biodiv...,,,...,Ninox,connivens,,2021-07-28T02:17:03Z,Ninox connivens,species,http://www.birdlife.org/datazone/speciesfactsh...,,,


In [376]:
# Stats
numsensitive = len(sensitivelist.index)
numconservation = len(conservationlist.index)
numupdates  = len(updates.index)
numadditions  = len(additions.index)
numnoinatstatus = len(noinatstatus.index)
numunknownToInat = len(unknownToInat.index)
numnotaddedupdated = len(notaddedupdated.index)
numnoncomply = len(noncomply.index)
numcomply = len(statelist.index)
numdupinfo = len(dupinformation.index)
d = {'Sensitive': [numsensitive],
    'Conservation': [numconservation],
    'Statelist merge': [numfullstatelist],
    'Species iNat Comply' : [numcomply],
    'Species iNat non-Comply': [numnoncomply],
    'Duplicate Information': [numdupinfo],
    'Updates': [numupdates],
    'Additions': [numadditions],
    'Not added updated': [numnotaddedupdated],
    'No Inat Status': [numnoinatstatus],
    'Unknown to Inat': [numunknownToInat]}

statsdf = pd.DataFrame(data=d)
statsdf

,Sensitive,Conservation,Statelist merge,Species iNat Comply,Species iNat non-Comply,Duplicate Information,Updates,Additions,Not added updated,No Inat Status,Unknown to Inat
0,251,1172,1172,1156,16,0,97,899,29,1059,160


In [294]:
# # last minute fudge
# sa = pd.read_csv(sourcedir + "sa.csv")
# sa['iucn_equivalent'] = sa['iucn_equivalent'].apply(lambda x: "Vulnerable" if (x == "Rare") else x)
# sa['place_id'] = "6899"
# sa['authority'] = "South Australia Department for Environment and Water"
# sa.to_csv(sourcedir + "sa.csv")